In [1]:
from model import Model
from multiprocessing.dummy import Pool
from functools import partial
import os

In [2]:
model = Model('./russian-syntagrus-ud-2.4-190531.udpipe')

In [26]:
def get_conllu(model, text):
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu = model.write(sentences, "conllu")
    return conllu

def save_result(idx, model):
    with open(f'../../data/texts/{idx}') as f:
        text = f.read()
    conllu = get_conllu(model, text)
    with open(f'../../data/conllu/{idx}', 'w') as f:
        f.write(conllu)
    return idx

In [29]:
partial_func = partial(save_result, model=model)
idxs = [i for i in os.listdir('../../data/texts/') if not i.startswith('.')]

In [30]:
%%time

pool = Pool(10)
results = pool.map(partial_func, idxs)
pool.close()
pool.join()

CPU times: user 259 ms, sys: 8 ms, total: 267 ms
Wall time: 332 ms
